# Анализ: Ridge, Quantile regression и Polynomial regression

Данный ноутбук содержит три блока:
1. `blood.xlsx` — Ridge регрессия (поиск alpha с графиками, VIF, проверка значимости, сравнение с OLS).
2. `Kuiper.xls` — квантильная регрессия для разных tau, визуализация и сравнение с OLS/медианной регрессией.
3. `cigarettes.dat.txt` — полиномиальные модели (квадратичная, кубическая) и сравнение с линейной регрессией.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.regression.quantile_regression import QuantReg

from sklearn.linear_model import Ridge, RidgeCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score

plt.rcParams['figure.figsize'] = (8,5)
sns.set(style='whitegrid')


## 1. `blood.xlsx` — Ridge regression

Переменные: `X1` = систолическое давление (зависимая переменная), `X2` = возраст, `X3` = вес (lbs).
Задачи:
- Построить ridge для X1 ~ X2 + X3; найти оптимальный alpha (k) и показать графики, аналогичные `cv.plot` и `bias.plot`.
- Проверить мультиколлинеарность (VIF).
- Проверить значимость уравнения регрессии.
- Сравнить ridge с OLS (MSE, R², AIC для OLS).


In [ ]:
bloodDf = pd.read_excel('../datasets/blood.xlsx')
display(bloodDf.head())
bloodDf = bloodDf.rename(columns=lambda s: s.strip())
yBlood = bloodDf['X1'].astype(float)
XBlood = bloodDf[['X2','X3']].astype(float)

### 1.2 Ridge — подбор alpha (k) и графики
Будем использовать `RidgeCV` для поиска alpha по кросс-валидации и построим два графика:
- `cv.plot`: средняя кросс-валидационная ошибка (MSE) vs alpha (log scale);
- `bias.plot`: поведение коэффициентов (и bias) vs alpha — покажем как меняются коэффициенты.

In [ ]:
scalerBlood = StandardScaler()
XBloodScaled = scalerBlood.fit_transform(XBlood)

alphaGrid = np.logspace(-4, 4, 1000)
ridgeCv = RidgeCV(alphas=alphaGrid, scoring='neg_mean_squared_error', store_cv_results=True)
ridgeCv.fit(XBloodScaled, yBlood)
bestAlpha = ridgeCv.alpha_
print('Best alpha (RidgeCV) =', bestAlpha)


# cv.plot: MSE vs alpha
meanMsePerAlpha = -np.mean(ridgeCv.cv_results_, axis=0)
plt.figure()
plt.semilogx(alphaGrid, meanMsePerAlpha)
plt.axvline(bestAlpha, color='red', linestyle='--', label=f'best alpha={bestAlpha:.4g}')
plt.xlabel('alpha (log scale)')
plt.ylabel('CV MSE')
plt.title('Ridge CV plot: alpha vs CV MSE')
plt.legend()
plt.show()

# bias.plot: коэффициенты vs alpha
coefsPerAlpha = []
for a in alphaGrid:
    ridgeTemp = Ridge(alpha=a)
    ridgeTemp.fit(XBloodScaled, yBlood)
    coefsPerAlpha.append(ridgeTemp.coef_)
coefsPerAlpha = np.array(coefsPerAlpha)

plt.figure()
for j, name in enumerate(XBlood.columns):
    plt.semilogx(alphaGrid, coefsPerAlpha[:, j], label=name)
plt.axvline(bestAlpha, color='red', linestyle='--')
plt.xlabel('alpha (log scale)')
plt.ylabel('Coefficient value')
plt.title('Ridge bias plot: coefficients vs alpha')
plt.legend()
plt.show()


### 1.3 Итоговая Ridge-модель и сравнение с OLS
Построим Ridge с лучшим alpha и OLS; сравним MSE, R² и — для OLS — AIC. Также оценим значимость модели: для OLS используем F-test; для Ridge применим permutation test (оценка значимости модели через перестановки).

In [ ]:
# Fit best Ridge
ridgeBest = Ridge(alpha=bestAlpha)
ridgeBest.fit(XBloodScaled, yBlood)
yPredRidge = ridgeBest.predict(XBloodScaled)
mseRidge = mean_squared_error(yBlood, yPredRidge)
r2Ridge = r2_score(yBlood, yPredRidge)

# Fit OLS via statsmodels to get summary and F-test
XBloodWithConst = sm.add_constant(XBlood)
olsModel = sm.OLS(yBlood, XBloodWithConst).fit()
yPredOls = olsModel.fittedvalues
mseOls = mean_squared_error(yBlood, yPredOls)
r2Ols = olsModel.rsquared

print('Ridge (alpha=%.6g): MSE=%.4f, R2=%.4f' % (bestAlpha, mseRidge, r2Ridge))
print('OLS: MSE=%.4f, R2=%.4f, AIC=%.4f' % (mseOls, r2Ols, olsModel.aic))
print('\nOLS summary:')
print(olsModel.summary())


In [ ]:
# Проверка значимости Ridge-модели через permutation test (R^2 на исходных данных vs случайные перемешивания y)
def permutationTestModelR2(Xmat, yvec, fitFunc, scoreFunc, nPerm=500, randomState=42):
    rng = np.random.RandomState(randomState)
    # fit on original
    model = fitFunc()
    model.fit(Xmat, yvec)
    scoreOrig = scoreFunc(model, Xmat, yvec)
    permScores = []
    for i in range(nPerm):
        yPerm = rng.permutation(yvec)
        modelPerm = fitFunc()
        modelPerm.fit(Xmat, yPerm)
        permScores.append(scoreFunc(modelPerm, Xmat, yPerm))
    permScores = np.array(permScores)
    pValue = (permScores >= scoreOrig).sum() / float(nPerm)
    return scoreOrig, permScores, pValue

def ridgeFitFactory(alpha=bestAlpha):
    return lambda: Ridge(alpha=alpha)

def r2ScoreFunc(model, Xmat, yvec):
    ypred = model.predict(Xmat)
    return r2_score(yvec, ypred)

scoreOrigRidge, permScoresRidge, pValRidge = permutationTestModelR2(XBloodScaled, yBlood, lambda: Ridge(alpha=bestAlpha), r2ScoreFunc, nPerm=500)
print('Ridge permutation test: observed R2=%.4f, permutation p-value=%.4f' % (scoreOrigRidge, pValRidge))

### 1.4 Мультиколлинеарность (VIF)
Вычислим VIF для предикторов X2, X3.

In [ ]:
vifDf = pd.DataFrame()
vifDf['feature'] = XBlood.columns
vifDf['VIF'] = [variance_inflation_factor(XBlood.values, i) for i in range(XBlood.shape[1])]
display(vifDf)

## 2. `Kuiper.xls` — Квантильная регрессия и сравнение с OLS

Переменные: `Price` (зависимая), `Mileage, Liter, Cruise, Sound, Leather` — предикторы.
Задачи:
- Построить квантильную регрессию для нескольких tau (например, 0.1, 0.25, 0.5, 0.75, 0.9).
- Изобразить коэффициенты vs tau и несколько прогнозных линий/эффектов.
- Построить OLS и сравнить с медианной регрессией (tau=0.5).


In [ ]:
kuiperDf = pd.read_excel('../datasets/Kuiper.xls')
display(kuiperDf.head())

def toFloatFromCommaSafe(x):
    try:
        if pd.isna(x):
            return np.nan
        return float(str(x).strip().replace(',', '.'))
    except:
        return np.nan

numColsKuiper = ['Price','Mileage','Liter','Cruise','Sound','Leather']
for c in numColsKuiper:
    if c in kuiperDf.columns:
        kuiperDf[c] = kuiperDf[c].apply(toFloatFromCommaSafe)

kuiperDfClean = kuiperDf.dropna(subset=numColsKuiper).copy()
yKuiper = kuiperDfClean['Price']
XKuiper = kuiperDfClean[['Mileage','Liter','Cruise','Sound','Leather']]


### 2.2 Квантильная регрессия для набора tau и визуализация коэффициентов


In [ ]:
taus = [0.1, 0.25, 0.5, 0.75, 0.9]
coefResults = {}
XKuiperWithConst = sm.add_constant(XKuiper)
for tau in taus:
    qrModel = QuantReg(yKuiper, XKuiperWithConst).fit(q=tau)
    coefResults[tau] = qrModel.params
    print(f'Quantile {tau}:')
    print(qrModel.summary())

coefDf = pd.DataFrame(coefResults).T
coefDf.index.name = 'tau'
display(coefDf)

# Plot coefficients vs tau
plt.figure()
for col in coefDf.columns:
    plt.plot(coefDf.index, coefDf[col], marker='o', label=col)
plt.xlabel('tau')
plt.ylabel('Coefficient')
plt.title('Quantile regression: coefficients vs tau')
plt.legend(bbox_to_anchor=(1.05,1), loc='upper left')
plt.show()


### 2.3 Сравнение OLS и медианной регрессии (tau=0.5)
Построим OLS и median (tau=0.5) и сравним прогнозы/ошибки (MSE, MAE).

In [ ]:
olsKuiperModel = sm.OLS(yKuiper, XKuiperWithConst).fit()
print('OLS summary:')
print(olsKuiperModel.summary())

medianCoef = coefResults[0.5]
medianPred = XKuiperWithConst.dot(medianCoef)

olsPred = olsKuiperModel.fittedvalues

mseOls = mean_squared_error(yKuiper, olsPred)
mseMedian = mean_squared_error(yKuiper, medianPred)
maeOls = np.mean(np.abs(yKuiper - olsPred))
maeMedian = np.mean(np.abs(yKuiper - medianPred))

print('Comparison on full sample:')
print('OLS: MSE=%.4f, MAE=%.4f' % (mseOls, maeOls))
print('Median (tau=0.5): MSE=%.4f, MAE=%.4f' % (mseMedian, maeMedian))


### 2.4 Визуализация: предсказанные квантильные линии для зависимости Price от Mileage (others at mean)


In [ ]:
plt.figure()
xVar = 'Mileage'
xGrid = np.linspace(XKuiper[xVar].min(), XKuiper[xVar].max(), 100)
meanVals = XKuiper.mean()
for tau in taus:
    coefTau = coefResults[tau]
    dfGrid = pd.DataFrame({
        'const': 1.0,
        'Mileage': xGrid,
        'Liter': meanVals['Liter'],
        'Cruise': meanVals['Cruise'],
        'Sound': meanVals['Sound'],
        'Leather': meanVals['Leather']
    })
    yGrid = dfGrid.dot(coefTau)
    plt.plot(xGrid, yGrid, label=f'tau={tau}')

plt.scatter(XKuiper[xVar], yKuiper, s=10, alpha=0.5)
plt.xlabel('Mileage')
plt.ylabel('Price')
plt.title('Quantile regression: Price vs Mileage (predicted quantiles)')
plt.legend()
plt.show()


## 3. `cigarettes.dat.txt` — нелинейные полиномиальные модели

Переменные: `carbon monoxide` (y), `tar`, `nicotine`, `weight` — x1,x2,x3.
Задачи:
- Построить полиномиальные модели степени 2 и 3 (включая кросс-термы) и OLS линейную модель.
- Сравнить модели по AIC, RMSE, R², и привести summary для каждой (через statsmodels OLS).


In [ ]:
rows = []
with open('../datasets/cigarettes.dat.txt', 'r', encoding='utf-8') as fh:
    for line in fh:
        tokens = line.strip().split()
        if len(tokens) < 4:
            continue
        numeric = tokens[-4:]
        brand = ' '.join(tokens[:-4])
        try:
            carbonMonoxideVal = float(numeric[0].replace(',', '.'))
            tarVal = float(numeric[1].replace(',', '.'))
            nicotineVal = float(numeric[2].replace(',', '.'))
            weightVal = float(numeric[3].replace(',', '.'))
            rows.append((brand, carbonMonoxideVal, tarVal, nicotineVal, weightVal))
        except:
            continue
cigDf = pd.DataFrame(rows, columns=['brand','carbonMonoxide','tar','nicotine','weight'])
display(cigDf.head())
cigDf = cigDf.dropna()
yCig = cigDf['carbonMonoxide']
XCig = cigDf[['tar','nicotine','weight']]

### 3.2 Линейная модель (OLS)


In [ ]:
XCigWithConst = sm.add_constant(XCig)
olsCigModel = sm.OLS(yCig, XCigWithConst).fit()
print('OLS Cigarettes summary:')
print(olsCigModel.summary())

### 3.3 Полиномиальные модели (квадрат и куб)
Построим design matrix с `PolynomialFeatures` и подгоним OLS через statsmodels, чтобы получить AIC и summary.

In [ ]:
def fitPolynomialOls(Xdf, yvec, degree):
    poly = PolynomialFeatures(degree=degree, include_bias=False)
    Xpoly = poly.fit_transform(Xdf)
    featureNames = poly.get_feature_names_out(Xdf.columns)
    XpolyDf = pd.DataFrame(Xpoly, columns=featureNames)
    XpolyWithConst = sm.add_constant(XpolyDf)
    model = sm.OLS(yvec, XpolyWithConst).fit()
    return model, XpolyWithConst, featureNames

modelsCig = {}
for deg in [2,3]:
    modelDeg, designDeg, featNamesDeg = fitPolynomialOls(XCig, yCig, deg)
    modelsCig[deg] = (modelDeg, designDeg, featNamesDeg)
    print(f'Polynomial degree={deg} summary:')
    print(modelDeg.summary())


### 3.4 Сравнение моделей (линейной, квадратичной, кубической)
Сравним по AIC, RMSE и R² (на обучающей выборке).

In [ ]:
comparisonRows = []
comparisonRows.append({'model':'OLS_linear','aic':olsCigModel.aic,'rmse':np.sqrt(mean_squared_error(yCig, olsCigModel.fittedvalues)),'r2':olsCigModel.rsquared})
for deg, (modelDeg, designDeg, featNamesDeg) in modelsCig.items():
    comparisonRows.append({'model':f'poly_{deg}','aic':modelDeg.aic,'rmse':np.sqrt(mean_squared_error(yCig, modelDeg.fittedvalues)),'r2':modelDeg.rsquared})
comparisonDf = pd.DataFrame(comparisonRows).set_index('model')
display(comparisonDf)

plt.figure()
minVal = min(yCig.min(), olsCigModel.fittedvalues.min())
maxVal = max(yCig.max(), olsCigModel.fittedvalues.max())
plt.plot([minVal, maxVal], [minVal, maxVal], '--', color='gray', label='y = x (perfect)')

plt.scatter(olsCigModel.fittedvalues, yCig, label='Linear fit', alpha=0.7)
for deg, (modelDeg, _, _) in modelsCig.items():
    plt.scatter(modelDeg.fittedvalues, yCig, label=f'Poly {deg}', alpha=0.6)
plt.xlabel('Predicted')
plt.ylabel('Observed carbonMonoxide')
plt.legend()
plt.title('Predicted vs Observed (comparison)')
plt.show()
